In [ ]:
#!pip install keras==2.8.0
#!pip install tensorflow==2.8.0

In [ ]:
from collections import deque
import numpy as np
from typing import Callable
import copy
from warnings import filterwarnings
import json
import os
from collections import defaultdict
import keras
import tensorflow as tf
from keras import backend as K
from scipy.special import softmax
from typing import List
from tqdm.notebook import tqdm
import random
import requests
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc = {'figure.figsize':(22,12)}, style="whitegrid")

filterwarnings('ignore')

In [ ]:
import pandas as pd
from sklearn.metrics import recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier

# 1. Policy Function & DQN Architicture
* 1- epsilon greedy implementaion for make action
* 2- DQN model and compile

In [ ]:
def epsilon_greedy(expected_reward, epsilon=0.97) -> int:
    """
    expected_reward: list of expected rewards for each possible action
    epsilon: .
    """
    if np.random.rand() <= epsilon:
        return np.random.choice(list(range(len(expected_reward))))
    else:
        return np.argmax(expected_reward)

PolicyFunction  = Callable[[np.ndarray, float], int]

In [ ]:
LEARNING_RATE = 0.001

def create_model(input_dim):
    K.clear_session()
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape=(input_dim,)))
    model.add(keras.layers.Dense(32, kernel_initializer='he_uniform', activation='relu'))
    model.add(keras.layers.Dense(16, kernel_initializer='he_uniform', activation='relu'))
    model.add(keras.layers.Dense(2))
    model.compile(loss='mse', optimizer='adam')
    return model

## 2. Agents Implementaion

* Agents class (Fair Agent Reward Distribution for Dynamic Feature Selection (FARD-DFS)) will be the parent of 4 types of agents listed as follow:
    * 1- Softmax (Alternative Contribution (ALC)) version (distrbute the total reward between agents using softmax function)
    * 2- Average (Accumulative Contribution (ACC)) version (distrbute the total reward between agents using average function)
    * 1- regression (Impurity (IM)) version (calcualte the contrbution of each agent using regression model)
    * 1- Single Agent (One Action at One Time (OA-OT)) version (only one agent at a time can make action and get the total reawrd as a result)

In [ ]:
class Agents:
    # class variable
    agent_count = 0

    def __init__(self, evaluation_network, buffer_size: int = 800):

        self.evaluation_network = evaluation_network
        self.target_network = copy.deepcopy(self.evaluation_network)
        self.buffer_size = buffer_size
        self.fitted = False
        self.agent_id = Agents.agent_count
        Agents.agent_count += 1
        # reply buffer is a list of tuples each tuples contains the following
        # (St, At, St+1, Rt+1)
        # (Current state, Action was made, New state, Reward)
        self.reply_buffer = deque(maxlen=self.buffer_size)
        self.contrbution = np.random.rand()

    def make_action(self, curr_state: np.ndarray, policy_function: PolicyFunction, epsilon) -> int:
        # q_values represents the expected rewards for each possible action
        if self.fitted:
            q_values = self.evaluation_network.predict(curr_state.reshape(-1, Agents.agent_count))
            action = policy_function(q_values, epsilon)
        else:
            action = policy_function([0, 1], 1)
        return action

    def update_target_network(self):
        self.target_network = copy.deepcopy(self.evaluation_network)
        return

In [ ]:
#Alternative Contribution (ALC) Fair Agent Reward Distribution for Dynamic Feature Selection (FARD-DFS)
class AgentsSoftmax(Agents):
    def __init__(self, evaluation_network, buffer_size=800):
        super().__init__(evaluation_network, buffer_size)


    def update_evaluation_network(self, batch_size=32, epochs=5, discount_factor=0.995):
        # select random batch from the reply buffer
        batch = random.sample(self.reply_buffer, batch_size)

        # inintilize some lists to store transition information
        Q1, actions, Q2, rewards = [], [], [], []

        # from each transition extract its values
        for transition in batch:
            Q1.append(transition[0])
            actions.append(transition[1])
            Q2.append(transition[2])
            rewards.append(transition[3])
        # X_train will be the states from
        X_train = np.array(Q1).reshape(-1, Agents.agent_count)

        expected_reward = self.evaluation_network.predict(np.array(Q1).reshape(-1, Agents.agent_count))
        Q2 = self.target_network.predict(np.array(Q2).reshape(-1, Agents.agent_count))

        # update expected rewards using biliman equation

        for i, act in enumerate(actions[:-1]):
            expected_reward[i, act] = rewards[i] + (discount_factor * np.argmax(Q2[i]))

        y_train = expected_reward.copy()

        # calculate the change frequency of the agent decision to use it as its contrbution in get total reward

        change_frequency = 0
        for state, next_state, reward, next_reward in zip(X_train[:-1], X_train[1:], rewards[: -1], rewards[1:]):
            #print(state, next_state, reward, next_reward)
            if np.abs(state[self.agent_id] - next_state[self.agent_id]) == 1:
                self.contrbution += np.abs(reward - next_reward)
                change_frequency += 1
        self.contrbution /= change_frequency

        # train the DQN evaluation network.
        self.evaluation_network.fit(X_train, y_train, epochs=epochs, verbose=0)
        self.fitted = True
        return

In [ ]:
# Impurity (IM) Fair Agent Reward Distribution for Dynamic Feature Selection (FARD-DFS)
class AgentsRegression(Agents):
    def __init__(self, evaluation_network, buffer_size=800):
        super().__init__(evaluation_network, buffer_size)

    def update_evaluation_network(self, batch_size=32, epochs=5, discount_factor=0.995):
        # select random batch from the reply buffer
        batch = random.sample(self.reply_buffer, batch_size)

        # inintilize some lists to store transition information
        Q1, actions, Q2, rewards = [], [], [], []

        # from each transition extract its values
        for transition in batch:
            Q1.append(transition[0])
            actions.append(transition[1])
            Q2.append(transition[2])
            rewards.append(transition[3])
        # X_train will be the states from
        X_train = np.array(Q1).reshape(-1, Agents.agent_count)

        expected_reward = self.evaluation_network.predict(np.array(Q1).reshape(-1, Agents.agent_count))
        Q2 = self.target_network.predict(np.array(Q2).reshape(-1, Agents.agent_count))

        # update expected rewards using biliman equation
        for i, act in enumerate(actions[:-1]):
            expected_reward[i, act] = rewards[i] + (discount_factor * np.argmax(Q2[i]))

        y_train = expected_reward.copy()
        # train the DQN evaluation network.
        self.evaluation_network.fit(X_train, y_train, epochs=epochs, verbose=0)
        self.fitted = True
        return

In [ ]:
# Accumulative Contribution (ACC) Fair Agent Reward Distribution for Dynamic Feature Selection (FARD-DFS)
class AgentsAverage(Agents):
    def __init__(self, evaluation_network, buffer_size=800):
        super().__init__(evaluation_network, buffer_size)

    def update_evaluation_network(self, batch_size=32, epochs=5, discount_factor=0.995):
        # select random batch from the reply buffer
        batch = random.sample(self.reply_buffer, batch_size)

        # inintilize some lists to store transition information
        Q1, actions, Q2, rewards = [], [], [], []

        # from each transition extract its values
        for transition in batch:
            Q1.append(transition[0])
            actions.append(transition[1])
            Q2.append(transition[2])
            rewards.append(transition[3])
        # X_train will be the states from
        X_train = np.array(Q1).reshape(-1, Agents.agent_count)

        expected_reward = self.evaluation_network.predict(np.array(Q1).reshape(-1, Agents.agent_count))
        Q2 = self.target_network.predict(np.array(Q2).reshape(-1, Agents.agent_count))

        # update expected rewards using biliman equation

        for i, act in enumerate(actions[:-1]):
            expected_reward[i, act] = rewards[i] + (discount_factor * np.argmax(Q2[i]))

        y_train = expected_reward.copy()

        WINDOW_SIZE = 4
        X_train_ = np.zeros((X_train.shape[0] // WINDOW_SIZE, X_train.shape[1]))
        y_train_ = []
        j = 0
        for i in range(0, batch_size, WINDOW_SIZE):
            window_of_states = X_train[i: i + WINDOW_SIZE].sum(axis=0) / WINDOW_SIZE
            window_of_rewards = sum(rewards[i: i + WINDOW_SIZE])
            r = window_of_rewards * window_of_states[self.agent_id]
             # Rounding state
            X_train_[j, :] = np.around(window_of_states)
            if window_of_states[self.agent_id] == 0:
                if window_of_rewards > 0.6:
                    r = window_of_rewards
                else:
                    r = window_of_rewards / WINDOW_SIZE
            y_train_.append(r)
            j += 1

        X_train = X_train_
        y_train = np.array(y_train_)
        # train the DQN evaluation network.
        self.evaluation_network.fit(X_train, y_train, epochs=epochs, verbose=0)
        self.fitted = True
        return

In [ ]:
# One Action at One Time (OA-OT) method of Fair Agent Reward Distribution for Dynamic Feature Selection (FARD-DFS)
class AgentsSingle(Agents):
    def __init__(self, evaluation_network, buffer_size=800):
        super().__init__(evaluation_network, buffer_size)

    def update_evaluation_network(self, batch_size=32, epochs=5, discount_factor=0.995):
        # select random batch from the reply buffer
        batch = random.sample(self.reply_buffer, batch_size)

        # inintilize some lists to store transition information
        Q1, actions, Q2, rewards = [], [], [], []

        # from each transition extract its values
        for transition in batch:
            Q1.append(transition[0])
            actions.append(transition[1])
            Q2.append(transition[2])
            rewards.append(transition[3])
        # X_train will be the states from
        X_train = np.array(Q1).reshape(-1, Agents.agent_count)

        expected_reward = self.evaluation_network.predict(np.array(Q1).reshape(-1, Agents.agent_count))
        Q2 = self.target_network.predict(np.array(Q2).reshape(-1, Agents.agent_count))

        # update expected rewards using biliman equation
        for i, act in enumerate(actions[:-1]):
            expected_reward[i, act] = rewards[i] + (discount_factor * np.argmax(Q2[i]))

        y_train = expected_reward.copy()
        # train the DQN evaluation network.
        self.evaluation_network.fit(X_train, y_train, epochs=epochs, verbose=0)
        self.fitted = True
        return

# 3. Reward Calculation Method
* 1- get accuracy of selected feature using logistic regression model
* 2- claculate the reward with reward_strategy (ACC, ALC, IM, and OA-OT of FARD-DFS) function using accuracy from last step.

In [ ]:
# test set percentage
TESTSIZE=0.2

def get_reward(dataset:pd.DataFrame, subset_features:list, y_label:str):
    global TESTSIZE
    # index of selected features
    subset_features = np.where(np.array(subset_features) == 1)[0]
    if subset_features.shape[0] == 0:return 0
    # train test split
    X, Y = dataset[subset_features.tolist()], dataset[y_label]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=TESTSIZE)
    # data normalization.
    std = StandardScaler()
    X_train = std.fit_transform(X_train)
    X_test = std.transform(X_test)
    # classification and model evaluation
    classifier = LogisticRegression(max_iter=500)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return acc

In [ ]:
def reward_strategy(time_step: int, accuracy: float, accuracy_history: list, subset_features: list, error_rate: float,
                    beta: float = 0.99):
    if sum(subset_features) == len(subset_features):
        return -5
    elif accuracy > max(accuracy_history):
        return 0.5
    elif accuracy < max(accuracy_history):
        return -0.1
    else:
        return -1 * (beta * error_rate + ((1 - beta) * (sum(subset_features) / len(subset_features))))

# 4. Some helper function
* 1- Object to store all information about the result like dataset name, accuracy, precision and etc...
* 2- Telegram api to send the result to a chat room
* 3- Send result to the chat room

In [ ]:
import pickle

class Results:
    def __init__(self, method_name, dataset_name, chunk_id, feature_space):
        self.dataset_name = dataset_name
        self.method_name = method_name
        self.chunk_id = chunk_id
        self.feature_space = feature_space
        self.feature_space_size = len(feature_space)
        self.result_information = {}

    def set_chunk_id(self, chunk_id: int):
        self.chunk_id = chunk_id

    def set_feature_space(self, feature_space: list):
        self.feature_space = feature_space
        self.feature_space_size = len(feature_space)

    def add_result(self, model_type:str, result:dict):
        self.result_information[model_type] = result

    def save(self, path='feature_selection_results'):
        file_name = self.method_name + '_' + self.dataset_name + '_' + '{}'.format(self.chunk_id) + '.pkl'
        with open(os.path.join(path, file_name), 'wb') as file_:
            pickle.dump(self, file_, pickle.HIGHEST_PROTOCOL)
        return

In [ ]:
class Telegram:

    def __init__(self, bot_token):
        self.end_point = 'https://api.telegram.org/bot'
        self.token = bot_token
        self.full_endpoint = self.end_point + self.token + '/'

    def __repr__(self):
        return 'your token is {}'.format(self.full_endpoint)

    def send_message(self, chat_id, message):
        send_text = self.full_endpoint + 'sendMessage'
        data = {'chat_id': chat_id, 'text': message}
        response = requests.get(send_text, data=data)
        return response

    def send_photo(self, chat_id, photo):
        url = self.full_endpoint + 'sendPhoto'
        data = {'chat_id': chat_id}
        files = {'photo': open(photo, 'rb')}
        response = requests.post(url, data=data, files=files)
        return response

    def send_document(self, chat_id, file):
        url = self.full_endpoint + 'sendDocument'
        data = {'chat_id': chat_id}
        files = {'document':open(file, 'rb')}
        response = requests.post(url, data = data, files = files)
        return response
    def get_updates(self):
        url = self.full_endpoint + 'getUpdates'
        response = requests.get(url)
        return response

    def get_file_information(self, file_id):
        url = f'https://api.telegram.org/bot{self.token}/getFile'
        response = requests.post(url,data = {"file_id":file_id})
        if response.status_code != 200:
            return {"ok":"False"}
        json_response = response.json()
        if json_response['ok'] == False:
            return {"ok":"False"}
        file_path = json_response['result']['file_path']
        file_information = requests.get(f'https://api.telegram.org/file/bot{self.token}/{file_path}')
        return file_information.text

In [ ]:
def send_results(telegram_api, results):
    telegram.send_message(1021388563, 'dataset name : {}'.format(results.dataset_name))
    telegram.send_message(1021388563, 'chunk id : {}'.format(results.chunk_id))
    telegram.send_message(1021388563, 'selected features : {}'.format(results.feature_space))
    telegram.send_message(1021388563, 'results')
    for key in  results.result_information.keys():
        telegram.send_message(1021388563, 'model tpye : {}'.format(key))
        telegram.send_message(1021388563, '{}'.format(results.result_information[key]))

# 5. Feature Selection Main Algorithm
* For each type of agents we need a specific feature selection algorithm with little difference between them.
* Evaluation function using different machine learning model based on selected feature
* Result Visulization

In [ ]:
def softmax_distrbution(agents):
    contrbutions = []
    for agent in agents:
        contrbutions.append(agent.contrbution)
    return softmax(contrbutions)


def random_forest_distrbution(dataset, num_of_samples=10000):
    X = []
    y = []
    for _ in range(num_of_samples):
        features_space = np.random.choice([0, 1], size=(num_of_agents,)).tolist()
        accuracy = get_reward(dataset, features_space, 'target')
        X.append(features_space)
        y.append(accuracy)
    X = np.array(X)
    y = np.array(y)
    rf = RandomForestRegressor(n_estimators=15)
    rf.fit(X, y)
    return rf.feature_importances_.tolist()

In [ ]:
def feature_selection(algo_type:str, agents: list, dataset: pd.DataFrame, y_label: str, eposide=200):
    """
    """
    column_names = list(range(dataset.shape[1]))
    column_names[-1] = 'target'
    dataset.columns = column_names
    epsilon = 0.01
    features_space = []
    NUMBER_OF_AGENTS = dataset.shape[1] - 1

    # get contrbutions
    contrbutions = []
    if algo_type == 'random_forest':
        contrbutions = random_forest_distrbution(dataset)
    elif algo_type in ['single_agent', 'average']:
        contrbutions = [1] * NUMBER_OF_AGENTS

    for i in tqdm(range(eposide)):
        # define the initial space
        features_space = np.random.choice([0, 1], size=(NUMBER_OF_AGENTS,)).tolist()
        # rewards history
        rewards = [0]

        # get action of each agent to create new feature space
        next_feature_space = []
        # contrbution of each agent

        if algo_type == 'softmax':
            contrbutions = softmax_distrbution(agents)

        for t in range(0, NUMBER_OF_AGENTS):
            action = agents[t].make_action(np.array(features_space.copy()), epsilon_greedy, epsilon)
            next_feature_space.append(action)
            if algo_type == 'single_agent':
                features_space[t] = action


        # calculate the total accuracy of new state (new feature space) and distrbute it using softmax

        # 1- get the accuracy using machine learning model trained in the current subset feature
        reward_as_accuracy = get_reward(dataset, next_feature_space, y_label)

        # 2- using the reward strategy map the accuracy value (reward_as_accuracy) to new reward value (reward_at_time_t)
        reward_at_time_t = reward_strategy(t, reward_as_accuracy, rewards, next_feature_space, 1 - reward_as_accuracy)
        # add the accuray of machine learning model to rewards list to use it in the mapping reward strategy.
        rewards.append(reward_as_accuracy)

        # total reward = reward after mapping
        total_reward = reward_at_time_t



        # add state and actions to agent buffer reply  and the reward which equals to contrbution of the agent*total reward
        transition = []
        for t in range(0, NUMBER_OF_AGENTS):
            transition.clear()
            # add current state (current feature space )
            feature_space_copy = features_space.copy()
            transition.append(feature_space_copy)
            # add agent's action to the transition
            action = next_feature_space[t]
            transition.append(action)
            # add new state (new feature space) into transition
            transition.append(next_feature_space)
            # add distrbuted reward to the transition
            transition.append(total_reward * contrbutions[t])
            # add the transition to reply buffer
            agents[t].reply_buffer.append(transition)
            if len(agents[t].reply_buffer) > 32 and i % 32 == 0:
                agents[t].update_evaluation_network()
        if i % 64 == 0:
            for agent in agents:
                if agent.fitted:
                    agent.update_target_network()
        epsilon = 0.97 * epsilon
    return next_feature_space

In [ ]:
def model_evaluation(method_name, dataset_name, dataset, selected_features, target_column, chunk_id, telegram_api):
    models = [
              ('SVM', svm.SVC(kernel='rbf', max_iter=8000, C=0.2, probability=True)),
              ('KNN', KNeighborsClassifier(5)),
              ('DecisionTree', DecisionTreeClassifier(random_state=42)),
              ('RandomForest', RandomForestClassifier()),
              ('LogRegression', LogisticRegression(max_iter=500))
     ]
    # index of selected features
    subset_features = np.where(np.array(selected_features) == 1)[0]
    if subset_features.shape[0] == 0:return 0
    # train test split
    X, Y = dataset[subset_features.tolist()], dataset[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=TESTSIZE)
    # data normalization.
    std = StandardScaler()
    X_train = std.fit_transform(X_train)
    X_test = std.transform(X_test)
    # classification and model evaluation
    results = Results(method_name, dataset_name, chunk_id, subset_features.tolist())
    for model in models:
        model_name, model_obj = model[0], model[1]
        model_obj.fit(X_train, y_train)
        y_pred = model_obj.predict(X_test)
        acc, p, r, f1, auc = accuracy_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred), roc_auc_score(y_test, y_pred)
        results.add_result(model_name, {'accuracy': acc, 'precision': p, 'recall': r, 'f1-score': f1, 'AUC': auc})
    send_results(telegram_api, results)
    results.save()
    print(results.result_information)

In [ ]:
def main(filenames, telegram_api, algorithm_type):
    # store the results
    for f_name in filenames:
        # read csv file
        df = pd.read_csv('dataset/{}'.format(f_name))
        column_names = list(range(df.shape[1]))
        column_names[-1] = 'target'
        df.columns = column_names
        # number of row in the dataframe
        number_of_samples = df.shape[0]
        # define the cunck size
        CHUNK_SIZE = number_of_samples // 10
        #####################################################
        chunk_id = 1
        for i in range(0, df.shape[0], CHUNK_SIZE):
            if chunk_id > 9:
                break
            chunk = df.iloc[i: i + CHUNK_SIZE, :]
            X, Y = chunk.drop('target', axis=1), chunk['target']
            agents = []
            #AgentsRegression.agent_count = 0
            #AgentsSoftmax.agent_count = 0
            #AgentsSingle.agent_count = 0
            #AgentsAverage.agent_count = 0
            Agents.agent_count = 0
            NUM_OF_FEATURES = df.shape[1] - 1 #  -1 for the target column


            for i in range(NUM_OF_FEATURES):
                if algorithm_type == 'softmax':
                    agents.append(AgentsSoftmax(create_model(NUM_OF_FEATURES)))
                elif algorithm_type == 'average':
                    agents.append(AgentsAverage(create_model(NUM_OF_FEATURES)))
                elif algorithm_type == 'single_agent':
                    agents.append(AgentsSingle(create_model(NUM_OF_FEATURES)))
                elif algorithm_type == 'random_forest':
                    agents.append(AgentsRegression(create_model(NUM_OF_FEATURES)))

            result = feature_selection(algorithm_type, agents, chunk, 'target')
            model_evaluation(algorithm_type, f_name, chunk, result, 'target', chunk_id, telegram_api)
            chunk_id += 1

In [ ]:
telegram = Telegram('5328835953:AAEisAKmtBl9TfGOFHQW0KaXcynsdRo7OKE')
filenames = os.listdir('dataset/')[::-1]
filenames

['syn_data.csv',
 'ppdaidata.csv',
 'IramTariq_Dmoz_xssed.csv',
 'IramTariq_DataTakenFromYunZhou.csv',
 'germandata.csv',
 'defaultdata.csv',
 'Data_167_featurs.csv']

In [ ]:
indices_to_access = [2, 3, 6]

accessed_mapping = map(filenames.__getitem__, indices_to_access)
filenames = list(accessed_mapping)
filenames

['IramTariq_Dmoz_xssed.csv',
 'IramTariq_DataTakenFromYunZhou.csv',
 'Data_167_featurs.csv']

In [ ]:
import sys

for algo in ['softmax', 'average', 'single_agent', 'random_forest']:
    main(filenames, telegram, algo)

INFO:tensorflow:Assets written to: ram://e1167f88-5f61-407b-8536-70de958b21e9/assets
INFO:tensorflow:Assets written to: ram://0ae7db15-ab31-425e-8e48-a2a734a52a9a/assets
INFO:tensorflow:Assets written to: ram://991c9f32-22a3-4187-98e3-3fc94748227f/assets
INFO:tensorflow:Assets written to: ram://5a36448d-0c43-4a10-9533-9546f7a4a5ef/assets
INFO:tensorflow:Assets written to: ram://016efd1e-00e2-4c4c-9e9f-1460fc9f9a8e/assets
INFO:tensorflow:Assets written to: ram://5bfd547d-659a-48ca-b72a-8838877c0163/assets
INFO:tensorflow:Assets written to: ram://f9a6a42c-e0f6-4d7b-8c59-a9d6d73f3aa9/assets
INFO:tensorflow:Assets written to: ram://a9fc6741-1c31-4c14-b4d8-f32e63891f16/assets
INFO:tensorflow:Assets written to: ram://46b3b3f3-15cc-43cf-a1eb-9a0d1c01cc3c/assets
INFO:tensorflow:Assets written to: ram://6903ad6d-0ca9-4142-a8e3-eaaa59b116c6/assets
INFO:tensorflow:Assets written to: ram://f1e8337a-94f4-4189-9473-95602f814ce4/assets
INFO:tensorflow:Assets written to: ram://51a9f820-bcb1-4e82-bcee-

  0%|          | 0/200 [00:00<?, ?it/s]

INFO:tensorflow:Assets written to: ram://a612b16e-7632-4565-a452-603aec87e18f/assets
INFO:tensorflow:Assets written to: ram://8fe6a948-21af-4af5-9110-8a7e7744e001/assets
INFO:tensorflow:Assets written to: ram://7d78a3c0-2218-4be8-945a-924f317b309e/assets
INFO:tensorflow:Assets written to: ram://08c36607-5c26-4d5f-a44c-82e41e7e4ba6/assets
INFO:tensorflow:Assets written to: ram://48e10fb4-f7e0-4476-a77b-f45edb0e1948/assets
INFO:tensorflow:Assets written to: ram://971ea30f-e4a2-4129-80de-13f0e3350caa/assets
INFO:tensorflow:Assets written to: ram://98dd4851-d268-463a-8444-c8a6e4405602/assets
INFO:tensorflow:Assets written to: ram://90ce869f-4486-4e60-87b0-ea4cf6ab6411/assets
INFO:tensorflow:Assets written to: ram://5162026d-4494-45d7-b3e6-0504b3a6cdf4/assets
INFO:tensorflow:Assets written to: ram://48702ac0-3f8a-4bb7-813a-6cc25ec4b22f/assets
INFO:tensorflow:Assets written to: ram://55873f0d-dcab-4e64-8cf3-540d4e87e8e9/assets
INFO:tensorflow:Assets written to: ram://60f017d3-c6e7-4594-9a03-

  0%|          | 0/200 [00:00<?, ?it/s]

INFO:tensorflow:Assets written to: ram://aaaedf90-0fee-488f-9bf2-1dd21f49a8c7/assets
INFO:tensorflow:Assets written to: ram://334d8e95-b08f-4833-b57f-cb04b628593d/assets
INFO:tensorflow:Assets written to: ram://595fab77-862b-42d2-a850-de22860adade/assets
INFO:tensorflow:Assets written to: ram://604b4426-507b-4459-a182-75bac8ac57d4/assets
INFO:tensorflow:Assets written to: ram://537ee260-20e1-4ec5-a8c1-6c13fff20332/assets
INFO:tensorflow:Assets written to: ram://cdb2711d-72c9-40e6-87b8-934c14348c6a/assets
INFO:tensorflow:Assets written to: ram://c1fdd2f0-f103-4041-8f36-5c60e22fdf02/assets
INFO:tensorflow:Assets written to: ram://9c35bbcb-2768-404e-aef6-6dc48aaf6340/assets
INFO:tensorflow:Assets written to: ram://2c2752e9-dbd9-449b-a9f7-dce65aa79a02/assets
INFO:tensorflow:Assets written to: ram://21240e3a-5fb8-48a6-a74a-39fa73223bef/assets
INFO:tensorflow:Assets written to: ram://7b926d94-3ec9-41a6-86aa-2a1b49531659/assets
INFO:tensorflow:Assets written to: ram://8bebb615-918d-43e4-acfe-

...

INFO:tensorflow:Assets written to: ram://636a6316-ee44-4cbf-bc44-53c27885bcce/assets
INFO:tensorflow:Assets written to: ram://6adbd0c5-056a-4fb0-b272-5c058a37611e/assets
INFO:tensorflow:Assets written to: ram://9932bbc2-7e95-4c06-8986-ef0ba9a08aa1/assets
INFO:tensorflow:Assets written to: ram://8e1f2c62-710d-4d03-95d2-efccd7c3d5ad/assets
INFO:tensorflow:Assets written to: ram://2c5bcb76-2c79-4ca4-a409-23a5f9710182/assets
INFO:tensorflow:Assets written to: ram://6371337a-6fc8-470c-8cc9-73de22cdb398/assets
INFO:tensorflow:Assets written to: ram://78f69c1f-8241-4bc1-afe2-43e4d8861b9a/assets
INFO:tensorflow:Assets written to: ram://e893ea96-9a9a-46ea-ab37-ea45693efc9c/assets
INFO:tensorflow:Assets written to: ram://201f665f-df29-45b8-8d2c-0c9490f012a7/assets
INFO:tensorflow:Assets written to: ram://8082d5ae-4be0-4e53-b2a4-7d754ba7818c/assets
INFO:tensorflow:Assets written to: ram://c9cce80b-7f54-4649-ab43-1d3986114570/assets
INFO:tensorflow:Assets written to: ram://f8a4a21c-4755-4c90-92ff-

  0%|          | 0/200 [00:00<?, ?it/s]

INFO:tensorflow:Assets written to: ram://efe33bcb-28f4-4f6b-938d-220da4a68a7f/assets
INFO:tensorflow:Assets written to: ram://7586c019-4eb7-4e07-933b-d4398616d366/assets
INFO:tensorflow:Assets written to: ram://f92d4f9d-f9bc-4feb-8946-c740c80deb7b/assets
INFO:tensorflow:Assets written to: ram://89e2bc1c-ba29-4839-ad51-048fbddc8f26/assets
INFO:tensorflow:Assets written to: ram://0f36ec65-9da0-4813-ae0d-eed5ffc7333a/assets
INFO:tensorflow:Assets written to: ram://a700557e-1ca1-47d1-87c2-91987a820b48/assets
INFO:tensorflow:Assets written to: ram://669a638d-43bc-4a87-b614-3798426738b4/assets
INFO:tensorflow:Assets written to: ram://55686352-07d5-431d-88f5-f50d696442d1/assets
INFO:tensorflow:Assets written to: ram://bca2ec7d-4de3-48b5-a874-6295efdfc97a/assets
INFO:tensorflow:Assets written to: ram://ea3ad231-ad6b-4fe3-b607-dc339a374110/assets
INFO:tensorflow:Assets written to: ram://d92d529e-7016-45bc-a584-85e5231c0eb7/assets
INFO:tensorflow:Assets written to: ram://ea9fff13-47af-4db3-b70e-

  0%|          | 0/200 [00:00<?, ?it/s]

INFO:tensorflow:Assets written to: ram://2e673ccc-e8a1-4a09-baaf-a76000636586/assets
INFO:tensorflow:Assets written to: ram://c357a385-d0b5-4280-99c1-73fce36bbf86/assets
INFO:tensorflow:Assets written to: ram://9b81688a-6d97-472d-978e-e4134df0f56b/assets
INFO:tensorflow:Assets written to: ram://575402cd-7d39-40b1-b3e3-67b3b77c0df7/assets
INFO:tensorflow:Assets written to: ram://ee1a6bd5-deaa-4ba0-8855-cad2786d533e/assets
INFO:tensorflow:Assets written to: ram://2176e1d4-f566-4cd8-a9cd-22db837932f3/assets
INFO:tensorflow:Assets written to: ram://acf15ff9-a2dd-4344-a118-50a769d5b151/assets
INFO:tensorflow:Assets written to: ram://e08bd5e9-b473-4426-b605-40945f0aa0ab/assets
INFO:tensorflow:Assets written to: ram://12634b55-9c5a-4827-8fed-9a68af48d496/assets
INFO:tensorflow:Assets written to: ram://66e7a6a4-4291-45ef-9644-fad92977359a/assets
INFO:tensorflow:Assets written to: ram://50b0cbdf-199f-4217-91ca-a21b8d72c988/assets
INFO:tensorflow:Assets written to: ram://59409c5b-bd0b-4e96-8002-

In [ ]:
result_files = os.listdir('feature_selection_results')
#result_files = list(map(lambda x: os.path.join('feature_selection_results', x), result_files))

In [ ]:
MODEL_NAMES = ['SVM', 'KNN', 'DecisionTree', 'LogRegression']
METRICES = ['accuracy', 'precision', 'recall', 'f1-score', 'AUC']
METHOD_NAMES = []
DATASET_NAMES = []
RESULTS = []

for r_file in result_files:
    with open(os.path.join('feature_selection_results', r_file), 'rb') as f:
        r = pickle.load(f)
        if r_file[0] in ['I' ,'D']:
            METHOD_NAMES.append('Impurity')
        elif re.search(r'(softmax_)|(average_)|(single_agent_)', r_file):
            METHOD_NAMES.append(r.method_name)
        else:
            continue
        if re.search(r'(Data_167_featurs)|(IramTariq_DataTakenFromYunZhou)|(IramTariq_Dmoz_xssed)|(syn_data)', r.dataset_name):
            DATASET_NAMES.append(r.dataset_name[:-4])
            for model_name in MODEL_NAMES:
                model_results = []
                for metrice in METRICES:
                    model_results.append(r.result_information[model_name][metrice])
                model_results.append(model_name)
                model_results.append(METHOD_NAMES[-1])
                model_results.append(DATASET_NAMES[-1])
                RESULTS.append(model_results)

In [ ]:
tuples = list(zip(METHOD_NAMES, DATASET_NAMES))
index = pd.MultiIndex.from_tuples(tuples, names=["Method Name", "Dataset Name"])
index

MultiIndex([('average',               'Data_167_featurs.csv_1'),
            ('average',               'Data_167_featurs.csv_2'),
            ('average',               'Data_167_featurs.csv_3'),
            ('average',               'Data_167_featurs.csv_4'),
            ('average',               'Data_167_featurs.csv_5'),
            ('average',               'Data_167_featurs.csv_6'),
            ('average',               'Data_167_featurs.csv_7'),
            ('average',               'Data_167_featurs.csv_8'),
            ('average',               'Data_167_featurs.csv_9'),
            ('average', 'IramTariq_DataTakenFromYunZhou.csv_1'),
            ...
            ('softmax', 'IramTariq_DataTakenFromYunZhou.csv_9'),
            ('softmax',           'IramTariq_Dmoz_xssed.csv_1'),
            ('softmax',           'IramTariq_Dmoz_xssed.csv_2'),
            ('softmax',           'IramTariq_Dmoz_xssed.csv_3'),
            ('softmax',           'IramTariq_Dmoz_xssed.csv_4'),
         

In [ ]:
iterables =[MODEL_NAMES, METRICES]
cols = pd.MultiIndex.from_product(iterables, names=["Model name", "Metrices"])
cols

MultiIndex([(          'SVM',  'accuracy'),
            (          'SVM', 'precision'),
            (          'SVM',    'recall'),
            (          'SVM',  'f1-score'),
            (          'SVM',       'AUC'),
            (          'KNN',  'accuracy'),
            (          'KNN', 'precision'),
            (          'KNN',    'recall'),
            (          'KNN',  'f1-score'),
            (          'KNN',       'AUC'),
            ( 'DecisionTree',  'accuracy'),
            ( 'DecisionTree', 'precision'),
            ( 'DecisionTree',    'recall'),
            ( 'DecisionTree',  'f1-score'),
            ( 'DecisionTree',       'AUC'),
            ('LogRegression',  'accuracy'),
            ('LogRegression', 'precision'),
            ('LogRegression',    'recall'),
            ('LogRegression',  'f1-score'),
            ('LogRegression',       'AUC')],
           names=['Model name', 'Metrices'])

## 2. Feature Selection using RandomForestTree

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel


rft_results = defaultdict(list)

for f_name in filenames:
    # read csv file
    df = pd.read_csv('dataset/{}'.format(f_name))
    column_names = list(range(df.shape[1]))
    column_names[-1] = 'target'
    df.columns = column_names
    # number of row in the dataframe
    number_of_samples = df.shape[0]
    # define the cunck size
    CHUNK_SIZE = number_of_samples // 10
    chunk_id = 0
    #####################################################
    for i in range(0, df.shape[0], CHUNK_SIZE):
        if chunk_id > 9:
                break
        chunk = df.iloc[i: i+CHUNK_SIZE, :]
        X, Y = chunk.drop('target', axis=1), chunk['target']
        X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.2)
        sel = SelectFromModel(RandomForestClassifier(n_estimators=8, max_depth=5))
        sel.fit(X_train, y_train)
        selected_feat= X_train.columns[(sel.get_support())]
        feature_space = np.zeros((X.shape[1]), dtype=np.uint8)
        feature_space[selected_feat.tolist()] = 1
        rft_results['{}'.format(f_name)].append(get_reward(chunk, feature_space.tolist(), 'target'))
        chunk_id += 1

In [ ]:
def barplot(data, title, y, hue='method_name'):
    ax = sns.catplot(x='datasetname', y=y, hue=hue, data=df_results, row='model_name', kind='bar',
           height=3, aspect=4)
    sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
    plt.title(title)
    plt.savefig('{}'.format(title))
    plt.show()

In [ ]:
idx = 0
for d_name in df_results['datasetname'].unique().tolist():
    for m in METRICES:
        barplot(df_results[df_results['datasetname'] == d_name], '{}_{}_{}'.format(d_name, m, idx), y=m)
        idx += 1